In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from optbinning import OptimalBinning
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

from sklearn.preprocessing import Normalizer

In [2]:
dataset = pd.read_csv('creditcard.csv')
dataset

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:
#TARGET Distribution

a = dataset['Class'].value_counts().reset_index().rename(columns={'index':'transaction','Class':'count'})

# a.at[0, 'transaction'] = 'Non-fraudulent'
# a.at[1, 'transaction'] = 'Fraudulent'

a

,transaction,count
0,0,284315
1,1,492


In [4]:
#Excluding TIME AND AMOUNT COLUMNS FOR Features, we will have - 
X = dataset[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']]
y = dataset['Class']

In [5]:
X


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533


In [6]:
y

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64

In [7]:
list_features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']

In [8]:
X_list = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']
y_list = ['target']

## Building Models

### TEST TRAIN SPLIT

In [9]:
# Split dataset into training and test sets (80/20 split)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training set into training and validation sets (60/40 split)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

In [10]:
train = X_train.copy()
train['target'] = y_train

val = X_val.copy()
val['target'] = y_val

test = X_test.copy()
test['target'] = y_test

In [11]:
test.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
43428,-16.526507,8.584972,-18.649853,9.505594,-13.793819,-2.832404,-16.701694,7.517344,-8.507059,-14.110184,...,-1.514923,1.190739,-1.127670,-2.358579,0.673461,-1.413700,-0.462762,-2.018575,-1.042804,1
49906,0.339812,-2.743745,-0.134070,-1.385729,-1.451413,1.015887,-0.524379,0.224060,0.899746,-0.565012,...,0.506044,-0.213436,-0.942525,-0.526819,-1.156992,0.311211,-0.746647,0.040996,0.102038,0
29474,1.399590,-0.590701,0.168619,-1.029950,-0.539806,0.040444,-0.712567,0.002299,-0.971747,0.756801,...,0.212877,0.102398,0.168269,-0.166639,-0.810250,0.505083,-0.232340,0.011409,0.004634,0
276481,-0.432071,1.647895,-1.669361,-0.349504,0.785785,-0.630647,0.276990,0.586025,-0.484715,-1.376648,...,-0.244633,0.358932,0.873663,-0.178642,-0.017171,-0.207392,-0.157756,-0.237386,0.001934,0
278846,2.014160,-0.137394,-1.015839,0.327269,-0.182179,-0.956571,0.043241,-0.160746,0.363241,0.259452,...,-0.255293,-0.238644,-0.616400,0.347045,0.061561,-0.360196,0.174730,-0.078043,-0.070571,0


## Feature Scaling - Normalization

In [12]:
# create the normalizer object
normalizer = Normalizer()

In [13]:
train[X_list] = normalizer.fit_transform(train[X_list])

In [14]:
train[X_list].head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
201477,0.478607,-0.082935,-0.036688,0.389233,-0.221963,-0.193207,-0.100124,0.002558,0.278783,0.047490,...,-0.165498,-0.074979,-0.046297,-0.148017,0.109659,0.059918,-0.135830,-0.261521,0.009955,-0.004534
210354,-0.254960,0.338616,0.469523,-0.111454,-0.099812,-0.126510,0.055844,0.143613,-0.021584,-0.400578,...,0.106968,-0.031665,-0.020119,-0.066247,-0.010174,-0.033375,-0.095306,-0.180371,0.014348,0.027587
218330,-0.174122,0.213499,0.432969,0.396479,0.153376,0.164426,0.094412,0.056316,-0.247583,0.031273,...,-0.386733,-0.013668,0.030506,0.067260,-0.073339,0.098400,0.124064,0.007371,0.002614,0.012946
237957,-0.172083,-0.711283,-0.291611,0.058685,-0.243956,0.089663,0.164451,-0.039843,-0.034167,-0.015104,...,0.071095,0.398333,0.099184,-0.154281,-0.135575,0.025515,-0.110173,-0.076754,-0.036623,0.029065
121958,0.351653,-0.223809,0.110296,-0.199202,-0.294862,-0.181656,-0.143256,-0.053627,-0.213767,0.154847,...,0.192075,0.096308,0.007880,-0.066877,-0.016980,-0.031104,0.082790,-0.112090,0.000347,0.011004


In [15]:
train.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
count,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,...,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000,170883.000000
mean,0.058331,0.017022,0.027245,0.002646,-0.002671,-0.028468,0.015248,0.006272,-0.002553,-0.012338,...,-0.003875,-0.003504,-0.006264,-0.000475,-0.000527,-0.001001,0.001749,0.000727,0.001481,0.001849
std,0.331978,0.234664,0.289483,0.275448,0.220466,0.239076,0.183673,0.133219,0.213067,0.174044,...,0.084482,0.087457,0.165163,0.070896,0.137782,0.117453,0.114486,0.057159,0.037583,0.042963
min,-0.840497,-0.743414,-0.703062,-0.745800,-0.706935,-0.640857,-0.577676,-0.871554,-0.691331,-0.566400,...,-0.540089,-0.413817,-0.624060,-0.849516,-0.573584,-0.713873,-0.424577,-0.563131,-0.357114,0.000000
25%,-0.209440,-0.138146,-0.186524,-0.201992,-0.164191,-0.193544,-0.122964,-0.052123,-0.148077,-0.128480,...,-0.052234,-0.055767,-0.127261,-0.037969,-0.080933,-0.076519,-0.074541,-0.017624,-0.011966,0.000000
50%,0.003863,0.016445,0.044293,-0.005606,-0.013911,-0.066181,0.010091,0.005659,-0.012258,-0.024857,...,-0.015626,-0.006775,0.001280,-0.002539,0.009699,0.002978,-0.010024,0.000311,0.002847,0.000000
75%,0.361928,0.189468,0.245599,0.184161,0.145974,0.088308,0.137623,0.075220,0.140998,0.093928,...,0.032134,0.042361,0.111921,0.034941,0.102037,0.077708,0.058462,0.020236,0.017311,0.000000
max,0.708086,0.605971,0.835362,0.799311,0.789486,0.867285,0.772505,0.440629,0.702013,0.680627,...,0.416330,0.635446,0.555318,0.517606,0.810685,0.578288,0.668663,0.293793,0.422655,1.000000


In [16]:
val[X_list] = normalizer.transform(val[X_list])
val.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
104878,0.473638,0.069600,0.076959,0.512181,0.034264,0.039995,0.027420,0.004912,0.128491,-0.050881,...,-0.063790,-0.051962,-0.059589,-0.052042,-0.166547,0.290366,-0.112577,0.015228,0.004817,0
105552,0.268726,-0.052799,-0.245892,-0.247018,0.377494,0.628968,-0.123739,0.154498,0.030021,-0.028037,...,0.014016,-0.028739,-0.110720,-0.001471,0.214462,0.082390,0.199683,-0.016306,-0.000373,0
9128,-0.159395,0.288831,0.304283,0.013615,0.011854,-0.152204,0.142105,-0.039892,0.252344,0.026182,...,0.074981,-0.097215,-0.161629,0.004347,0.083001,-0.036273,-0.002771,0.054287,-0.003162,0
201800,-0.159243,-0.000943,-0.047979,-0.509154,0.069090,-0.239631,0.022252,0.021276,0.279282,-0.168533,...,-0.041017,0.008339,0.039586,0.094498,0.158979,-0.489595,-0.060740,-0.003451,0.037000,0
60023,0.023191,0.362526,0.077129,0.349087,0.104600,-0.245503,0.193346,-0.030050,-0.104526,-0.154679,...,-0.036286,0.056065,0.184157,0.031006,0.081843,-0.316168,-0.136812,0.033152,0.060497,0


In [17]:
val.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
count,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,...,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000,56962.000000
mean,0.058166,0.016533,0.026845,0.004136,-0.002357,-0.028960,0.015872,0.006451,-0.002994,-0.012768,...,-0.003968,-0.003340,-0.007007,-0.000570,-0.001234,-0.000323,0.001578,0.001200,0.001674,0.001369
std,0.332121,0.234711,0.289435,0.275819,0.220794,0.237655,0.183112,0.131869,0.214171,0.174067,...,0.084492,0.087157,0.165087,0.071901,0.138627,0.116659,0.114389,0.057114,0.037651,0.036979
min,-0.789484,-0.750409,-0.702807,-0.734481,-0.685529,-0.607418,-0.547219,-0.829412,-0.708847,-0.548689,...,-0.516623,-0.392681,-0.591770,-0.844311,-0.572748,-0.672393,-0.384536,-0.567941,-0.305857,0.000000
25%,-0.211574,-0.139115,-0.187058,-0.200824,-0.164232,-0.192932,-0.122492,-0.051698,-0.149666,-0.128798,...,-0.052160,-0.055234,-0.128266,-0.038338,-0.082613,-0.075467,-0.074608,-0.017414,-0.011826,0.000000
50%,0.006578,0.015510,0.045459,-0.002722,-0.012971,-0.066721,0.010874,0.005334,-0.013203,-0.024903,...,-0.015736,-0.006688,0.000901,-0.002304,0.009089,0.003741,-0.010630,0.000304,0.002884,0.000000
75%,0.360098,0.188864,0.245961,0.186067,0.145634,0.087143,0.136459,0.074932,0.139767,0.093213,...,0.032134,0.041926,0.111232,0.035176,0.102213,0.077319,0.058707,0.020542,0.017637,0.000000
max,0.695002,0.600500,0.834398,0.783265,0.776062,0.832314,0.768191,0.437230,0.666044,0.678738,...,0.428555,0.636402,0.557543,0.511785,0.826935,0.635244,0.693624,0.251197,0.473650,1.000000


In [ ]:
test[X_list] = normalizer.transform(test[X_list])
test.head()

In [ ]:
g = sns.FacetGrid(train[X_list].melt(), col='variable', col_wrap=4, sharex=False, sharey=False, height=4)
g.map(sns.histplot, 'value', bins=30)

In [ ]:
b = train[X_list].describe()
pd.set_option('float_format', '{:.17f}'.format)
b

## ALGORITHMS

### 1. Logistic Regression

In [ ]:
log_reg = LogisticRegression(random_state = 0)
log_reg.fit(train[X_list], train[y_list])

In [ ]:
train['prediction_logreg'] = log_reg.predict(train[X_list])
train.head(5)

In [ ]:
val['prediction_logreg'] = log_reg.predict(val[X_list])
val.head(5)

In [ ]:
test['prediction_logreg'] = log_reg.predict(test[X_list])
test.head(5)

In [ ]:
cm_lg_tr = confusion_matrix(train['target'], train['prediction_logreg'])
print(cm_lg_tr)

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cm_lg_tr), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix - Logistic Reg (train)', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
cm_lg_val = confusion_matrix(val['target'], val['prediction_logreg'])
print(cm_lg_val)

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cm_lg_val), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix - Logistic Reg (Validation)', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
cm_lg_ts = confusion_matrix(test['target'], test['prediction_logreg'])
print(cm_lg_ts)

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cm_lg_ts), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix - Logistic Reg (TEST)', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
def calculate_metrics(df, x):
    acc = accuracy_score(df['target'], df[x])
    prec = precision_score(df['target'], df[x])
    rec = recall_score(df['target'], df[x])
    f1 = f1_score(df['target'], df[x])
    auc = roc_auc_score(df['target'], df[x])
    
    print(f" Accuracy is {acc}")
    print(f" Precision is {prec:2f}")
    print(f" Recall is {rec:2f}")
    print(f" F1-Score is {f1:2f}")
    print(f" AUC score is {auc:2f}")


In [ ]:
calculate_metrics(train, 'prediction_logreg')

In [ ]:
calculate_metrics(val, 'prediction_logreg')

In [ ]:
calculate_metrics(test, 'prediction_logreg')

## 2. K Neighbours

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15)

In [ ]:
# train the classifier using the training set
knn.fit(train[X_list], train[y_list])

### 2.1 - KNN ON TRAIN SET -

In [ ]:
# make predictions on the Train set
train['prediction_KNN'] = knn.predict(train[X_list])

In [ ]:
cm_knn_tr = confusion_matrix(train['target'], train['prediction_KNN'])
cm_knn_tr

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cm_knn_tr), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('KNN - Confusion matrix - Train', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')


In [ ]:
calculate_metrics(train, 'prediction_KNN')